In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

## Objetivo de este cuadernillo

En este cuadernillo se quiere identificar la canasta de procedimientos que se le hace a cada uno de los diagnósticos distintos. Por ejemplo:

- A un paciente de Cáncer de Pulmón, cuál es la canasta de procedimientos que se le realiza?
- Dentro de un procedimiento (Ej: Radiografía de Tórax), cuántos de estos se les realizará?

Para esto se utilizará la base de procedimientos de Datos Estadísticos. Esta base contiene todos los procedimientos con código MAI del INT. Sin embargo, esta base carece del diágnostico de cada paciente. Por lo tanto, para solventar este problema se realizará lo siguiente:

1. Asignar último diagnóstico registrado de un paciente por GRD
2. Asignar último diagnóstico registrado de un paciente por HIS
3. Calcular desempeño de cada procedimiento por cada uno de los diagnósticos

Esto significará que se obtendrá la siguiente información:

|Año|Diagnóstico|Procedimiento|N° Pacientes Distintos|Procedimientos Totales|Distribución Proced.|
|---|---|---|---|---|---|
|2020|C. de Pulmón|Radiografía de Tórax|560|897|...|
|2020|C. de Pulmón|TAC|740|950|...|
|...|...|...|...|...|...|

Posteriormente, será necesario identificar el % de pacientes que requiere cada uno de los procedimientos. Por ejemplo:

- Para los pacientes de C.de pulmón, solamente el 35% de todo el universo de C. de Pulmón requiere Radiografía de Tórax.

Con esta información, el flujo de información será el siguiente:

Pacientes de Área de Influencia para uno de los diagnósticos &rarr; Obtener el pool de pacientes que requerira cada uno de los procedimientos &rarr; Aplicar el desempeño de cada uno de los procedimienots respectivos

In [ ]:
# Lee base de consultas y egresos
df_grd = pd.read_csv("../data/processed/grd_interno.csv")
df_grd["fecha_de_egreso"] = pd.to_datetime(df_grd["fecha_de_egreso"])
df_grd = df_grd.sort_values("fecha_de_egreso")

df_his = pd.read_csv("../data/external/his_procesada.csv")
df_his["fecha_atencion"] = pd.to_datetime(df_his["fecha_atencion"])
df_his = df_his.sort_values("fecha_atencion")

# Lee base de procedimientos
df_procedimientos = pd.read_csv("../data/external/procedimientos_procesada.csv")

## Asignacion de Diagnostico a cada procedimiento

In [ ]:
# Aisla los diagnosticos de cada una de las bases de datos
diags_grd = df_grd.groupby("RUT_LIMPIO").tail(1)[["diag_01_principal_cod", "diag_01_principal_des"]]
diags_grd = diags_grd.reset_index(names="id_paciente")

diags_his = df_his.groupby("id_paciente").tail(1)[["codigo_diagnostico", "nombre_diagnostico"]]
diags_his = diags_his.reset_index(names="id_paciente")

In [ ]:
# Agrega diagnosticos a los procedimientos
procedimientos_con_diags = df_procedimientos.merge(diags_his, how="left", on="id_paciente")